# The Cogs of Blackmail

In a city where steam and gears govern the world, a major steampunk company named SteamVault Industries dominates the market for mechanical inventions. You are a cybersecurity expert serving the citizens of this futuristic city.

One morning, Victor Gearstone, the chief system administrator of SteamVault, receives an anonymous message from a hacker claiming to possess compromising images of his wife. In exchange for silence, the hacker demands that Victor grant them permanent access to the company’s internal system, jeopardizing critical industrial secrets. Panicked, Victor turns to you to investigate and determine whether the hacker’s claims are genuine or a sophisticated lie.

Your mission is to inspect the received files, analyze the malware sent by the hacker, and uncover the truth. Everything must be done with precision and discretion.

sha256 : ae50d86ac083698999075e8cdf4aac49ff6f1f93622753b04a5cd122155bcd06

## Observations:

We have a C# dll, after decompiling we see it contains an aes payload decrypt it using key and iv and then execute a method on the decrypted payload.
There is also lots of junk code to confuse the analyst. 

Our goal: decompile and parse these files using a script to get the next compiled dll and so on until we get the final payload.

In [ ]:
import os, json
from Crypto.Cipher import AES

def dump_dll(i):
    os.system(f'ilspycmd dlls/{i}.dll -p -o data/{i}')

def get_correct_file(i):
    for f in os.listdir(f'data/{i}'):
        if f.endswith('.cs'):
            return f"data/{i}/{f}"


def parse_file(path):
    d = {}
    with open(path, 'r') as f:
        lines = f.read().split(';')
        parser = CSParser(lines)
        d = parser.parse()
    key = parseval(d['byte[] key'])
    iv = parseval(d['byte[] iV'])
    array2 = parseval(d['array2'])
    
    return {
        'key': key,
        'iv': iv,
        'array2': array2
    }

def decode(i, val):
    key = val['key']
    iv = val['iv']
    encrypted_data = val['array2']
    
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = cipher.decrypt(encrypted_data)
    with open(f'dlls/{i+1}.dll', 'wb') as f:
        f.write(decrypted_data)

def parseval(val):
    val = val.split('{')[1].split('}')[0]
    val = [
        int(v.strip())
        for v in val.split(',')]

    return bytes(val)

class CSParser:
    def __init__(self, lines):
        self.lines = lines
        self.d = {}
        self.idx = 0
    
    def parse(self):
        while self.idx < len(self.lines):
            self.parse_line()
            self.idx += 1
            
        return self.d

    def parse_line(self):
        line = self.lines[self.idx].strip()
        if ' = ' in line:
            self.parse_assignment()
    
    def parse_assignment(self):
        line = self.lines[self.idx].strip()
        name, value = line.split(' = ')
        self.d[name] = value
# dump_dll(1)

def do(i):
    dump_dll(i)
    d = parse_file(get_correct_file(i))

    decode(i, d)

for i in range(1, 1000):
    print(i)
    do(i)

At the step 150, we have the flag in plaintext.